In [2]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader,random_split
from torchvision import transforms, datasets, models
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean = [0.4914, 0.4822, 0.4465] 
std = [0.2470, 0.2435, 0.2616] 
batch_size = 40
n_epochs = 50

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    # Random augmentations
    transforms.RandomRotation(40),  # Randomly rotate images by 40 degrees
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05),  # Random color jitter
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=mean, std=std)  # Normalize with mean and std
])

path='train'
all_train = datasets.ImageFolder(root = path, transform = train_transform)
train_size = int(0.9 * len(all_train))
validation_size = len(all_train) - train_size
train_dataset, validation_dataset = random_split(all_train , [train_size, validation_size])
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)
val_loader = DataLoader(
    validation_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)

In [6]:
image_size = (3, 224, 224)  # Example: 3 channels, 32x32 pixels
num_classes = 100

In [10]:
class ClassificationModel(nn.Module):
    def __init__(self, image_size, num_classes):
        super(ClassificationModel, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(image_size[0], 32, kernel_size=3, stride=1, padding='same') # (224x224x3) -> (224x224x16)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding='same') # (112x112x16) -> (112x112x32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='same')
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding='same')  # (56x56x32) -> (56x56x64)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2) # Halves the input dimensions
        self.dropout = nn.Dropout(0.5)
        
        # Fully connected layers
        self.fc1 = nn.Linear(14 * 14 * 128, 256) # 64 * 28 * 28 is the flattened feature map size after pooling
        self.fc2 = nn.Linear(256, 100) # 100 categories (classes)
        
    def forward(self, x):
        # Apply convolution layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        
        # Flatten the feature maps for the fully connected layer
        x = x.view(-1, 128 * 14 * 14)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [11]:

model = ClassificationModel(image_size, num_classes)
print(model)

ClassificationModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=25088, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=100, bias=True)
)


In [12]:
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=20,eta_min=1e-9)

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    
def train(model, train_loader, optimizer, loss_fn):
    model.train()
    train_loss = 0.
    corrects=0
    total = 0
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad() # step 1
        images = images.to(device)
        labels = labels.to(device)
        

        logits = model(images) # step 2 (forward pass)
        loss = loss_fn(logits, labels) # step 3 (compute loss)
        _, predictions = torch.max(logits, dim=1)
        corrects += predictions.eq(labels).sum().item()
        total += labels.size(0)
        
        loss.backward() # step 4 (backpropagation)
        optimizer.step()

        train_loss += loss.item()*images.size(0)
       
        
    train_loss = train_loss/len(train_loader.sampler)
    
    return train_loss, corrects/total    


@torch.no_grad()
def validate(model, valid_loader, loss_fn):
    model.eval()
    losses=0.
    corrects=0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader):
            images = images.to(device)
            labels = labels.to(device)
            
                
            logits = model(images) # step 2 (forward pass)
            loss = loss_fn(logits, labels) # step 3 (compute loss)
            total += labels.size(0)
            
            _, predictions = torch.max(logits, dim=1)
            corrects += predictions.eq(labels).sum().item()
            
            losses += loss.item()*images.size(0)    
            
        valid_loss = losses/len(valid_loader.sampler)
    return valid_loss, corrects / total

train_loss_list = []
valid_loss_list = []

early_stopper = EarlyStopper(patience=7)

for epoch in range(n_epochs):
    training_loss, training_accuracy = train(model, train_loader, optimizer, loss_fn)
    valid_loss, valid_accuracy = validate(model, val_loader, loss_fn)
    
    train_loss_list.append(training_loss)
    valid_loss_list.append(valid_loss)

    # if scheduler is not None and is_valid_available:
    #     scheduler.step(valid_loss)
    # elif scheduler is not None:
    print(f"Epoch {epoch+1}/{n_epochs}: training accuracy: {training_accuracy}, valid accuracy: {valid_accuracy},train loss: {training_loss},valid loss:{valid_loss}") 
    scheduler.step()

100%|██████████| 35/35 [00:09<00:00,  3.87it/s]


Epoch 1/50: training accuracy: 0.027636594663278273, valid accuracy: 0.07928571428571428,train loss: 4.342366316721431,valid loss:3.978481354032244


100%|██████████| 35/35 [00:09<00:00,  3.89it/s]


Epoch 2/50: training accuracy: 0.07870076238881829, valid accuracy: 0.12714285714285714,train loss: 3.9117657111287873,valid loss:3.7028652599879672


100%|██████████| 35/35 [00:09<00:00,  3.79it/s]


...

Epoch 48/50: training accuracy: 0.4645806861499365, valid accuracy: 0.4714285714285714,train loss: 1.9463670505198924,valid loss:2.011249954359872


100%|██████████| 35/35 [00:09<00:00,  3.74it/s]


Epoch 49/50: training accuracy: 0.4760959339263024, valid accuracy: 0.4835714285714286,train loss: 1.9186404156563697,valid loss:2.0255509206226896


100%|██████████| 35/35 [00:09<00:00,  3.75it/s]

Epoch 50/50: training accuracy: 0.4934879288437103, valid accuracy: 0.48428571428571426,train loss: 1.8588161433667059,valid loss:1.9562922171183994
